<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,RMSprop
import numpy as np
import pandas as pd
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [2]:
def load_dataset(img_folder):
  IMG_WIDTH=200
  IMG_HEIGHT=200
  x=[]
  y=[]
  for dir_name in os.listdir(img_folder):
    for file in os.listdir(os.path.join(img_folder, dir_name)):
      img_path = os.path.join(img_folder, dir_name, file)
      img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
      #image = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
      image = (np.array(img)).astype('float32')
      image /= 255
      x.append(image)
      y.append(classes.index(dir_name))
  return x, y

In [25]:
def load_data():
  X, Y = load_dataset('/content/drive/MyDrive/inaturalist_12K/train')
  x_test, y_test = load_dataset('/content/drive/MyDrive/inaturalist_12K/val')
  x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.1, stratify = Y)

  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = load_data()

In [ ]:
x_test, y_test = load_dataset('/content/drive/MyDrive/inaturalist_12K/val')

In [ ]:
def augmentation(x, y, frac_hor, frac_vert, frac_flip):
  num = int(frac_hor*x.shape[0])
  rand_num_arr_hor = np.random.randint(x.shape[0]-1,size =(num))
  image = x[rand_num_arr_hor]
  for img in image:
    flipped = tf.image.flip_left_right(img)
    x_new = x.append(flipped)
  y_new = y.append(y[rand_num_arr_hor])

In [ ]:
def CNN_wo_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, fdropout=0, dense_no=512, learning_rate=0.001):

  # Input layer
  X_input = keras.Input(shape=X_train.shape)

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def CNN_w_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, fdropout=0, dense_no=512, learning_rate=0.001):

  # Input layer
  X_input = keras.Input(shape=X_train.shape)

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [ ]:
def model_choose(X_train, highest_filter, filter_type, kernel_matrix, activation_matrix, dropout=0, dense_no=512, batch_norm=True, learning_rate=0.001):
  if filter_type == 'constant':
    filter_matrix = [filter_start]*4
  elif filter_type == 'doubling':
    filter_matrix = [highest_filter/16, highest_filter/8, highest_filter/4, highest_filter/2, highest_filter]
  elif filter_type == 'halving':
    filter_matrix = [highest_filter, highest_filter/2, highest_filter/4, highest_filter/8, highest_filter/16]
  # wandb.log({"Filters": filter_matrix})

  if batch_norm == True:
    model = CNN_w_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  else:
    model = CNN_wo_BN(X_train, filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  
  return model


In [ ]:
# Checking all above functions
X=tf.random.normal((227,227,3))
filter_matrix = [32, 64, 256, 512, 1024]
kernel_matrix = [(3,3)]*5
activation_matrix = ['relu']*6;
model = model_choose(X, 1024, 'doubling', kernel_matrix, activation_matrix)
model.summary()

In [ ]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient
# Here model is the best model
GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=8).output]) # Here the index is to be checked
layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

with tf.GradientTape() as tape:
  inputs = tf.cast(X_input, tf.float32)
  tape.watch(inputs)
  outputs = GBModel(inputs)

grads = tape.gradient(outputs,inputs)[0]